In [1]:
from math import sqrt
import scipy.stats
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import TTestIndPower
import numpy as np
import scipy.stats as stats

## Задача 1

Вы провели эксперимент c упрощением формы заказа в магазине Утконос и получили результаты по метрике конверсий в покупку. Выберите метод оценки и оцените есть ли стат.значимые различия между конверсиями в двух группах при alpha = 5%. Дайте краткие рекомендации команде .Результаты:
- Число юзеров в группах , которые заходили на сайт в период эксперимента: n1 = 15550 и n2 = 15550 . 
- Число юзеров в группах , которые совершили хотя бы одну покупку за период эксперимента: n1 = 164 и n2 = 228
- Конверсии : conv1 = 1.05% conv2 = 1.47% .

### Решение 1

In [2]:
n_users_1, n_users_2 = 15_550, 15_550
n_users_1_conv, n_users_2_conv = 164, 228
conv_1, conv_2 = n_users_1_conv/n_users_1, n_users_2_conv/n_users_2

In [3]:
var_1 = conv_1 * (1 - conv_1) / n_users_1
var_2 = conv_2 * (1 - conv_2) / n_users_2
left = np.round(
    (conv_2 - conv_1 - 1.96 * (var_1 + var_2) ** .5) * 100,
    3)
right = np.round(
    (conv_2 - conv_1 + 1.96 * (var_1 + var_2) ** .5) * 100,
    3)
ci = [f"{left}%", f"{right}%"]
print(ci)

['0.164%', '0.66%']


Методе оценки: метод доверительного интервала<br>
Если 0 не входит в доверительный интервал, то значит, на основе собранных данных между группами имеется статистически значимая разница, следовательно, принимаем Альтернативную гипотезу: (упрощение формы заказа в магазине Утконос).

### Решение 2 (с семинара)

In [4]:
def calc_z_di(total_control, total_test, n_control, n_test):
    
    сonv_control = n_control / total_control
    conv_test = n_test / total_test
    
    lift = (conv_test - сonv_control) / сonv_control
    
    SE_control = sqrt(сonv_control * (1 - сonv_control) / total_control)
    SE_test = sqrt(conv_test * (1 - conv_test) / total_test)  
    SE_diff = sqrt(SE_control ** 2 + SE_test ** 2)
    z_core = (conv_test - сonv_control) / SE_diff
    pvalue = scipy.stats.norm.sf(abs(z_core)) * 2
    
    left = conv_test - сonv_control - 1.96 * SE_diff
    right = conv_test - сonv_control + 1.96 * SE_diff

    result = { "lift" : lift,
     "se_control" : SE_control,
     "se_test" : SE_test,
     "z_core" : z_core,
     "p_value" : pvalue,
     "ci" : [left, right]}
     
    return result 
calc_z_di (total_control=15500,total_test=15500,n_control=164, n_test=228)

{'lift': 0.3902439024390243,
 'se_control': 0.0008218270404826454,
 'se_test': 0.0009669807620454625,
 'z_core': 3.253677121726018,
 'p_value': 0.0011392168150319737,
 'ci': [0.0016417223858688043, 0.006616342130260227]}

## Задача 2

Сравниваем метрику конверсия в покупку. Размер выборки - 10000 элементов в каждой группе.<br> 
Какой статистический критерий тут лучше всего подойдёт и почему ?

При решении задач сравнения конверсий в покупку с помощью χ²(хи-квадрат)-теста.<br>
Предпочтительнее использовать χ²-тест на независимость, проводимый по таблице сопряженности, поскольку он обеспечивает оптимальные результаты с точки зрения ошибок первого и второго рода при сравнении двух и более воронок.<br>
[Информацию взял тут](https://habr.com/ru/company/mygames/blog/677074/)